In [88]:
import cohere  #LLM used for the development of the application
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
import os
import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI, Request, HTTPException, Form
from langchain.prompts.few_shot import FewShotPromptTemplate #for creating prompts with few-shot examples
from langchain.prompts.prompt import PromptTemplate #for formatting the few-shot examples
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage
import json
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain_cohere import ChatCohere
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
import psycopg2 #type: ignore
import json

load_dotenv()
cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
DB_CONFIG = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
}

In [89]:
def get_db_connection():
    conn = psycopg2.connect(
        user=DB_CONFIG["user"],
        password=DB_CONFIG["password"],
        host=DB_CONFIG["host"],
        port=DB_CONFIG["port"],
    )
    return conn

In [90]:
### Convierte las preguntas y respuestas en un dataframe de python
def db_get_questions():
    conn = get_db_connection()
    cursor = conn.cursor()
    # query = ''' SELECT * FROM respuestas_usuarios '''
    query = '''
    SELECT pr.id_pregunta, pr.usuario, pr.contenido as pregunta_contenido, pr.categoria,
    rs.contenido as respuesta_contenido, rs.id_respuesta, rs.id_pregunta, pr.activa FROM preguntas as pr
    LEFT JOIN respuestas as rs on pr.id_pregunta = rs.id_pregunta
    '''
    # cursor.fetchall()
    preguntas = pd.read_sql(query,conn)
    print(cursor)
    conn.close()
    return preguntas
preguntas = db_get_questions()
preguntas

In [95]:
def db_user_context(id_sesion):
    conn = get_db_connection()
    cursor = conn.cursor()
    # query = ''' SELECT * FROM respuestas_usuarios '''
    cursor.execute('''
        SELECT *
        FROM respuestas_usuarios
        WHERE id_sesion = %s
        ORDER BY id_respuesta_usuario DESC LIMIT 1
        ''', (id_sesion,))
    desc = cursor.description
    column_names = [col[0] for col in desc]
    data = [dict(zip(column_names, row))  
            for row in cursor.fetchall()]
    conn.close()
    return data[0]
info_user= db_user_context("prueba_raul_limpiador")
info_user

In [5]:
categoria = "divulgacion"
tipo_usuario = "usuario"
df_preguntas = db_get_questions()
df_preguntas['respuesta_contenido'].tolist()

In [6]:
df_preguntas = db_get_questions()
lista_preguntas= df_preguntas[(df_preguntas['categoria']==categoria) & (df_preguntas['usuario']==tipo_usuario) & (df_preguntas['activa']==True)]['pregunta_contenido'].unique().tolist()
lista_respuestas = df_preguntas['respuesta_contenido'].tolist()


In [56]:
lista_preguntas

In [ ]:
def llm_limpiador(id_sesion, categoria):

    ### Sacamos contexto del usuario y las preguntas activas, antes de tocar por el LLM.
    info_user= db_user_context(id_sesion)
    id_respuesta_usuario=info_user['id_respuesta_usuario']
    direccion_ip=info_user['direccion_ip']
    tipo_usuario=info_user['tipo_usuario']
    municipio=info_user['municipio']
    ccaa=info_user['ccaa']
    conocer_felgtbi=info_user['conocer_felgtbi']
    data_date=info_user['data_date']
    vih_usuario=info_user['vih_usuario']
    vih_diagnostico=info_user['vih_diagnostico']
    vih_tratamiento=info_user['vih_tratamiento']
    us_edad=info_user['us_edad']
    us_pais_origen=info_user['us_pais_origen']
    us_genero=info_user['us_genero']
    us_orientacion=info_user['us_orientacion']
    us_situacion_afectiva=info_user['us_situacion_afectiva']
    us_hablado=info_user['us_hablado']
    pro_ambito=info_user['pro_ambito']
    pro_especialidad=info_user['pro_especialidad']
    pro_vih_profesional=info_user['pro_vih_profesional']

    ### Preguntas
    df_preguntas = db_get_questions()
    preguntas_raw = df_preguntas[(df_preguntas['categoria']==categoria) & (df_preguntas['usuario']==tipo_usuario) & (df_preguntas['activa']==True)]['pregunta_contenido'].unique().tolist()
    dict_preguntas_raw = {}
    for i, pregunta in enumerate(preguntas_raw):
        dict_preguntas_raw[i] = pregunta
    dict_preguntas_raw

    ### LLM
    cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
    llm = ChatCohere(cohere_api_key=cohere_api_key, temperature=0,model='command-r')

    ### Response Schema para parser
    response_schemas = [
        ResponseSchema(name="diccionario_preguntas", description= '''Preguntas finales, en formato diccionario de Python. La clave es el índice
                       de la pregunta original, y el valor la pregunta filtrada. Ejemplo: 0:(pregunta1), 1:(pregunta2)'''),
        ResponseSchema(name="message", description="Explica de forma muy concisa por qué has excluido o incluido cada pregunta.")
    ]
    output_parser = JsonOutputParser() 

    ### Template y ejecución
    template = ChatPromptTemplate([
        ("system", ''' Recibes una lista de preguntas en {preguntas}. Tu misión es filtrar esas preguntas en función del contexto que
         conozcas del usuario. Pregunta sobre el embarazo sólo a tipo Mujer, o mujer cis género, y pregunta sobre la próstata sólo a tipo Hombre, u hombre cis género.
        Si el usuario no tiene vih, elimina del listado todas las preguntas relativas al vih, bajo ninguna
         circunstancia debes preguntar sobre el vih si el usuario no tiene vih. Sólo debes preguntar sobre el vih si el usuario nos confirma que tiene vih.
        NUNCA, en ningún caso modifiques el texto de las preguntas. Sólo quitarás según consideres dado el contexto del usuario, pero NUNCA variarás su contenido.
         Siempre escribirás "vih" (siempre en minúsculas, nunca en mayúsculas), ya que estamos desetigmatizando sobre la enfermedad.
         Devuelve exactamente un JSON con dos claves: 
        - "diccionario_preguntas": Las preguntas finales como un diccionario de Python. La clave de la pregunta debe coincidir con el índice del listado de preguntas en {preguntas},
         mientras que el valor será el texto de la pregunta.
        - "message": Respuesta al usuario por parte del bot, explicando de forma detallada por qué has excluido o incluido cada pregunta.
        NUNCA, NUNCA, incluyas texto adicional fuera del JSON.
         '''),
        ("human", '''Soy tipo de usuario {tipo_usuario}.  Vivo en {municipio} en la comunidad {ccaa}. Tengo {us_edad} años y nací en {us_pais_origen}.
         Mi género es {us_genero} y mi orientación sexual es {us_orientacion}. Sobre mi situación afectiva, {us_situacion_afectiva}.
         {us_hablado} he hablado sobre si tengo vih con mi entorno. A la pregunta de si tengo vih, la respuesta es {vih_usuario}, y, en caso de tenerlo, me dieron
         el diagnóstico hace {vih_diagnostico}, y si sigo tratamiento es desde hace {vih_tratamiento}. Si soy tipo de usuario "profesional",
        mi ámbito es {pro_ambito}, mi especialidad {pro_especialidad} y {pro_vih_profesional} he tratado a un afectado por vih.
          ''')],
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
        )
    prompt_value = template.invoke({"preguntas":preguntas_raw,"tipo_usuario":tipo_usuario,"municipio":municipio, "ccaa":ccaa, "conocer_felgtbi":conocer_felgtbi, "vih_usuario":vih_usuario,
                                    "vih_diagnostico":vih_diagnostico, "vih_tratamiento":vih_tratamiento, "us_edad": us_edad, "us_pais_origen":us_genero,
                                    "us_genero":us_genero,"us_orientacion":us_orientacion, "us_situacion_afectiva": us_situacion_afectiva,
                                    "us_hablado":us_hablado,"pro_ambito":pro_ambito,"pro_especialidad":pro_especialidad,
                                    "pro_vih_profesional":pro_vih_profesional})
    
    print(dict_preguntas_raw)
    print(us_genero)
    print(vih_usuario)
    response = llm.invoke(prompt_value)
    print(response.content)
    parsed_output = output_parser.parse(response.content)
    

    return parsed_output


In [34]:
preguntas = llm_limpiador('prueba_raul_limpiador', 'divulgacion')
dict_preg_resp = {}
for pregunta in preguntas['diccionario_preguntas'].keys():
    resp_list = df_preguntas[df_preguntas["pregunta_contenido"] == pregunta]["respuesta_contenido"].tolist()
    dict_preg_resp[pregunta] = resp_list
dict_preg_resp

In [30]:
preguntas

In [48]:
dict_preg_resp = {}
for pregunta in preguntas['diccionario_preguntas'].keys():
    resp_list = df_preguntas[df_preguntas["pregunta_contenido"] == pregunta]["respuesta_contenido"].tolist()
    dict_preg_resp[pregunta] = resp_list
dict_preg_resp

In [20]:
preguntas_raw = df_preguntas[(df_preguntas['categoria']=="divulgacion") & (df_preguntas['usuario']=="usuario") & (df_preguntas['activa']==True)]['pregunta_contenido'].unique()
preguntas_raw
dict_preguntas_raw = {}
for i, pregunta in enumerate(preguntas_raw):
    dict_preguntas_raw[i] = pregunta
dict_preguntas_raw

### Ejemplo con Álex

In [ ]:
### Ejemplo de Álex
def llm_limpiador(id_sesion, categoria):
    # Obtener contexto del usuario
    info_user = db_user_context(id_sesion)
    tipo_usuario = info_user['tipo_usuario']
    municipio = info_user['municipio']
    ccaa = info_user['ccaa']
    vih_usuario = info_user['vih_usuario']
    us_edad = info_user['us_edad']
    us_pais_origen = info_user['us_pais_origen']
    us_genero = info_user['us_genero']
    us_orientacion = info_user['us_orientacion']
    us_situacion_afectiva = info_user['us_situacion_afectiva']
    us_hablado = info_user['us_hablado']
    pro_ambito = info_user['pro_ambito']
    pro_especialidad = info_user['pro_especialidad']
    pro_vih_profesional = info_user['pro_vih_profesional']

    # Obtener preguntas activas
    df_preguntas = db_get_questions()
    preguntas_unicas = df_preguntas[
        (df_preguntas['categoria'] == categoria) &
        (df_preguntas['usuario'] == tipo_usuario) &
        (df_preguntas['activa'] == True)
    ][['pregunta_contenido','id_pregunta']].drop_duplicates()
    lista_preguntas = preguntas_unicas['pregunta_contenido'].tolist()
    lista_ids = preguntas_unicas['id_pregunta'].iloc[:,0].tolist()
    dict_preguntas = {}
    for i in range(len(lista_preguntas)):
        dict_preguntas[lista_ids[i]] = lista_preguntas[i]
    dict_preguntas

    # Configuración del LLM
    cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
    llm = ChatCohere(cohere_api_key=cohere_api_key, temperature=0, model='command-r')

    # Template del prompt
    prompt = f"""
    Recibes un diccionario de preguntas: {dict_preguntas}. La clave es el ID de la pregunta, y el valor es el texto de la pregunta.
    Tu tarea es filtrar las preguntas de acuerdo con el contexto del usuario, siguiendo las siguientes reglas:

    1. **Filtrado por VIH**:
    - Si el usuario **NO tiene vih**, elimina todas las preguntas relacionadas con padecer dicha enfermedad.

    2. **Filtrado por género**:

        - **Género masculino**: Si el género del usuario es **masculino**, elimina preguntas que no sean relevantes para este género. Por ejemplo, si una pregunta está relacionada con aspectos específicos de la biología femenina (como embarazo, periodo, etc), elimínala. Es decir, solo mantén preguntas que sean pertinentes a la biología o situaciones de los hombres.

        - **Género femenino**: Si el género del usuario es **femenino**, elimina preguntas que no sean relevantes para este género. Por ejemplo, si una pregunta está relacionada con aspectos específicos de la biología masculina (como próstata), elimínala. Es decir, solo mantén preguntas que sean pertinentes a la biología o situaciones de las mujeres.

        - **Género no binaria**: Si el género del usuario es **no binario**, elimina preguntas que se refieran de manera exclusiva a biología o situaciones de los géneros masculinos o femeninos. Este género no se asocia con los aspectos tradicionales de los géneros binarios, por lo que solo se deben incluir preguntas de carácter general o que no dependan del género biológico.

        - **Prefiero no decirlo**: Si el usuario seleccionó **prefiero no decirlo** como género, elimina preguntas que estén asociadas con un género específico, ya que no se puede asumir el género del usuario. Solo incluye preguntas que no dependan de un género específico.

    **Importante**: No modifiques el texto de las preguntas. Solo elimina las preguntas según las reglas mencionadas. Nunca debes modificar el contenido de las preguntas, solo eliminarlas si no son relevantes según el contexto.

    **Importante**: No escribirás VIH en mayúsculas en ningún caso ya que estamos desestigmatizando dicha enfermedad.

    Información del usuario:
    Tipo: {tipo_usuario}, Municipio: {municipio}, CCAA: {ccaa}, Edad: {us_edad}, País de origen: {us_pais_origen},
    Género: {us_genero}, Orientación: {us_orientacion}, Situación afectiva: {us_situacion_afectiva}, Ha hablado sobre vih: {us_hablado},
    Tiene vih: {vih_usuario}, Profesional (ámbito: {pro_ambito}, especialidad: {pro_especialidad}, experiencia con vih: {pro_vih_profesional}).

    Devuelve un diccionario en Python con las preguntas filtradas, en este formato:
    {{
        "preguntas_filtradas": {{ id de la pregunta original: "texto de la pregunta" }},
        "message": "Explicación de por qué se incluyeron/excluyeron las preguntas."
    }}
    """

    # Invocar el modelo
    response = llm.invoke(prompt)
    print("Respuesta del modelo:", response.content)

    # Procesar la respuesta como un diccionario
    try:
        result = eval(response.content.strip())
    except (SyntaxError, ValueError) as e:
        print(f"Error procesando la respuesta: {e}")
        result = {
            "preguntas_filtradas": {},
            "message": "Error al procesar la respuesta del modelo. Verifica el prompt o el formato de salida."
        }

    return result


In [10]:
preguntas = llm_limpiador('prueba_raul_limpiador', 'divulgacion')

preguntas['preguntas_filtradas']

In [74]:
list(preguntas['preguntas_filtradas'].keys())

In [89]:
df_preguntas

In [134]:
claves = list(preguntas['preguntas_filtradas'].keys())

preguntas_reales_filtradas_pero_reales = list(df_preguntas[df_preguntas['id_pregunta'].iloc[: , 0].isin(claves)]['pregunta_contenido'].unique())

In [135]:
preguntas_reales_filtradas_pero_reales

In [142]:
df_preguntas['respuesta_contenido']

In [172]:
df_preguntas[list(df_preguntas["pregunta_contenido"].isin(preguntas_reales_filtradas_pero_reales))]['respuesta_contenido']

In [178]:
dict_espero_que_definitivo_o_me_cuelgo = {}

for pregunta in preguntas_reales_filtradas_pero_reales:

     respuestones = df_preguntas[df_preguntas["pregunta_contenido"] == pregunta]["respuesta_contenido"].tolist()


     dict_espero_que_definitivo_o_me_cuelgo[pregunta] = respuestones

dict_espero_que_definitivo_o_me_cuelgo

In [63]:
preguntas_unicas = df_preguntas[
        (df_preguntas['categoria'] == categoria) &
        (df_preguntas['usuario'] == tipo_usuario) &
        (df_preguntas['activa'] == True)
    ][['pregunta_contenido','id_pregunta']].drop_duplicates()
lista_preguntas = preguntas_unicas['pregunta_contenido']
type(lista_preguntas)

In [65]:
preguntas_unicas = df_preguntas[
        (df_preguntas['categoria'] == categoria) &
        (df_preguntas['usuario'] == tipo_usuario) &
        (df_preguntas['activa'] == True)
    ][['pregunta_contenido','id_pregunta']].drop_duplicates()
lista_preguntas = preguntas_unicas['pregunta_contenido'].tolist()
lista_ids = preguntas_unicas['id_pregunta'].iloc[:,0].tolist()
dict_preguntas = {}
for i in range(len(lista_preguntas)):
    dict_preguntas[lista_ids[i]] = lista_preguntas[i]
dict_preguntas


In [38]:
df_preguntas

In [44]:
pregunta1 = "¿Has notado algun sintoma desde el diagnostico?"
df_preguntas[df_preguntas["pregunta_contenido"] == pregunta1]["respuesta_contenido"].tolist()

In [9]:
df_preguntas = db_get_questions()
df_preguntas

In [57]:
dict_preg_resp = {}
for pregunta in df_preguntas["pregunta_contenido"].drop_duplicates():
     resp_list = df_preguntas[df_preguntas["pregunta_contenido"] == pregunta]["respuesta_contenido"].tolist()
     dict_preg_resp[pregunta] = resp_list
dict_preg_resp

In [ ]:
preguntas_raw

In [130]:
listado_preguntas = respuesta['listado_preguntas']

In [132]:
lista_preguntas[1]

In [ ]:
listta

In [91]:
def llm_prueba(id_sesion, categoria):
    ### Sacamos contexto del usuario y las preguntas activas, antes de tocar por el LLM.
    info_user= db_user_context(id_sesion)
    id_respuesta_usuario=info_user['id_respuesta_usuario']
    direccion_ip=info_user['direccion_ip']
    tipo_usuario=info_user['tipo_usuario']
    municipio=info_user['municipio']
    ccaa=info_user['ccaa']
    conocer_felgtbi=info_user['conocer_felgtbi']
    data_date=info_user['data_date']
    vih_usuario=info_user['vih_usuario']
    vih_diagnostico=info_user['vih_diagnostico']
    vih_tratamiento=info_user['vih_tratamiento']
    us_edad=info_user['us_edad']
    us_pais_origen=info_user['us_pais_origen']
    us_genero=info_user['us_genero']
    us_orientacion=info_user['us_orientacion']
    us_situacion_afectiva=info_user['us_situacion_afectiva']
    us_hablado=info_user['us_hablado']
    pro_ambito=info_user['pro_ambito']
    pro_especialidad=info_user['pro_especialidad']
    pro_vih_profesional=info_user['pro_vih_profesional']
    locals().update(info_user)
    print(id_respuesta_usuario)
    print(conocer_felgtbi)
    print(us_orientacion)
llm_prueba('prueba_raul_limpiador', 'divulgacion')


## ENCAPSULO EL LIMPIADOR ÁRBOL

In [92]:
def diccionario_final_arbol(preguntas):

     claves = eval(preguntas)

     preguntas_reales_filtradas_pero_reales = list(df_preguntas[df_preguntas['id_pregunta'].iloc[: , 0].isin(claves)]['pregunta_contenido'].unique())

     dict_espero_que_definitivo_o_me_cuelgo = {}

     for pregunta in preguntas_reales_filtradas_pero_reales:

          respuestones = df_preguntas[df_preguntas["pregunta_contenido"] == pregunta]["respuesta_contenido"].tolist()

          dict_espero_que_definitivo_o_me_cuelgo[pregunta] = respuestones

     return dict_espero_que_definitivo_o_me_cuelgo

In [104]:
### Ejemplo de Álex
def llm_limpiador(id_sesion, categoria):
     # Obtener contexto del usuario
     info_user = db_user_context(id_sesion)
     tipo_usuario = info_user['tipo_usuario']
     municipio = info_user['municipio']
     ccaa = info_user['ccaa']
     vih_usuario = info_user['vih_usuario']
     us_edad = info_user['us_edad']
     us_pais_origen = info_user['us_pais_origen']
     us_genero = info_user['us_genero']
     us_orientacion = info_user['us_orientacion']
     us_situacion_afectiva = info_user['us_situacion_afectiva']
     us_hablado = info_user['us_hablado']
     pro_ambito = info_user['pro_ambito']
     pro_especialidad = info_user['pro_especialidad']
     pro_vih_profesional = info_user['pro_vih_profesional']

     # Obtener preguntas activas
     df_preguntas = db_get_questions()
     preguntas_unicas = df_preguntas[
          (df_preguntas['categoria'] == categoria) &
          (df_preguntas['usuario'] == tipo_usuario) &
          (df_preguntas['activa'] == True)
     ][['pregunta_contenido','id_pregunta']].drop_duplicates()
     lista_preguntas = preguntas_unicas['pregunta_contenido'].tolist()
     lista_ids = preguntas_unicas['id_pregunta'].iloc[:,0].tolist()
     dict_preguntas = {}
     for i in range(len(lista_preguntas)):
          dict_preguntas[lista_ids[i]] = lista_preguntas[i]
     dict_preguntas

     # Configuración del LLM
     cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
     llm = ChatCohere(cohere_api_key=cohere_api_key, temperature=0, model='command-r-plus')

     # Template del prompt
     prompt = f"""
     Recibes un diccionario de preguntas: {dict_preguntas}. La clave es el ID de la pregunta, y el valor es el texto de la pregunta.
     Tu tarea es filtrar las preguntas de acuerdo con el contexto del usuario, siguiendo las siguientes reglas:

     1. **Filtrado por vih**:
     - Si el usuario **NO** tiene vih, elimina todas las preguntas relacionadas con padecer dicha enfermedad y pasas al filtrado de género.

     - Si el usuario **SI** tiene vih, lanzas las preguntas relacionadas con padecer dicha enfermedad y pasas al filtrado de género.

     2. **Filtrado por género**:

          - **Género masculino**: Si el género del usuario es **masculino**, elimina preguntas que no sean relevantes para este género. Por ejemplo, si una pregunta está relacionada con aspectos específicos de la biología femenina (como embarazo, periodo, etc), elimínala. Es decir, solo mantén preguntas que sean pertinentes a la biología o situaciones de los hombres.

          - **Género femenino**: Si el género del usuario es **femenino**, elimina preguntas que no sean relevantes para este género. Por ejemplo, si una pregunta está relacionada con aspectos específicos de la biología masculina (como próstata), elimínala. Es decir, solo mantén preguntas que sean pertinentes a la biología o situaciones de las mujeres.

          - **Género no binario**: Si el género del usuario es **no binario**, elimina preguntas que se refieran de manera exclusiva a biología o situaciones de los géneros masculinos o femeninos. Este género no se asocia con los aspectos tradicionales de los géneros binarios, por lo que solo se deben incluir preguntas de carácter general o que no dependan del género biológico.

          - **Prefiero no decirlo**: Si el usuario seleccionó **prefiero no decirlo** como género, elimina preguntas que estén asociadas con un género específico, ya que no se puede asumir el género del usuario. Solo incluye preguntas que no dependan de un género específico.

     **Importante**: No modifiques el texto de las preguntas. Solo elimina las preguntas según las reglas mencionadas. Nunca debes modificar el contenido de las preguntas, solo eliminarlas si no son relevantes según el contexto.

     **Importante**: No escribirás VIH en mayúsculas en ningún caso ya que estamos desestigmatizando dicha enfermedad.

     Información del usuario:
     Tipo: {tipo_usuario}, Municipio: {municipio}, CCAA: {ccaa}, Edad: {us_edad}, País de origen: {us_pais_origen},
     Género: {us_genero}, Orientación: {us_orientacion}, Situación afectiva: {us_situacion_afectiva}, Ha hablado sobre vih: {us_hablado},
     Tiene vih: {vih_usuario}, Profesional (ámbito: {pro_ambito}, especialidad: {pro_especialidad}, experiencia con vih: {pro_vih_profesional}).

     **Salida Esperada:**
     Devuelve única y exclusivamente una lista de Python con los ID de las preguntas filtradas seleccionadas. No debes devolver nada más, Solo una lista con los ID seleccionados.

          - **Ejemplo 1** [1, 4, 7]
          - **Ejemplo 2** [1, 3, 6, 8, 9]
     """

     # Invocar el modelo
     response = llm.invoke(prompt)
     # print("Respuesta del modelo:", response.content)


     return diccionario_final_arbol(response.content)

In [105]:
llm_limpiador('prueba_raul_limpiador', 'divulgacion')

## Pruebas encapsulamiento funciones

In [1]:
from model import llm_limpiador, db_user_interaction, db_user_context

In [2]:
llm_limpiador('prueba_raul_limpiador')

c:\Users\raulg\Documents\THEBRIDGE_DS\0.-Repo_Git\Desafio-Final\model.py:287: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preguntas = pd.read_sql(query,conn)


<cursor object at 0x00000244A8C718C0; closed: 0>
<cursor object at 0x00000244A8C73BC0; closed: 0>


c:\Users\raulg\Documents\THEBRIDGE_DS\0.-Repo_Git\Desafio-Final\model.py:287: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preguntas = pd.read_sql(query,conn)


{'¿Cuál es tu color favorito?': ['Rojo', 'Azul', 'Otro color'],
 '¿Las relaciones que mantuviste fueron con proteccion?': ['No', 'Si'],
 '¿Has notado síntomas desde que te diagnosticaron vih?': ['Si', 'No']}

In [3]:
id_sesion = 'prueba_raul_limpiador'
categoria_pregunta = 'divulgacion'
preguntas_limpias = {}

In [5]:
if len(preguntas_limpias) == 0:
    print("Recogiendo preguntas...")
    preguntas_limpias = llm_limpiador(id_sesion)
    print(len(preguntas_limpias))
else:
    print("Preguntas ya recogidas")

Preguntas ya recogidas


In [8]:
preguntas_limpias

{'¿Cuál es tu color favorito?': ['Rojo', 'Azul', 'Otro color'],
 '¿Las relaciones que mantuviste fueron con proteccion?': ['No', 'Si'],
 '¿Has notado síntomas desde que te diagnosticaron vih?': ['Si', 'No']}

In [2]:
db_user_interaction("prueba_raul_limpiador")

In [5]:
db_user_context("prueba_raul_limpiador")